Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import shapiro
import scipy.stats as stats
import pingouin as pg
import statistics
import plotly.graph_objects as go
import plotly.express as px 

In [ ]:
filepath = " " #path to folder containing logfiles for every tDCS conditon
filepaths = os.listdir(filepath)
order = [1, 2, 0]
filepaths = [filepaths[i] for i in order ]
# outliers = []
outliers = ['SUBC3FP2010', 'SUBC3FP2054']
blocks = ['Block'+str(i+1) for i in range(8)]
block_mean = pd.DataFrame(index = blocks)
block_norm_cent_mean = pd.DataFrame(index = blocks)
block_rt_abs_mean = pd.DataFrame(index = blocks)
block_std = pd.DataFrame(index = blocks)
block_rt_abs_std = pd.DataFrame(index = blocks)
block_rt_abs_er = pd.DataFrame(index = blocks)
block_err = pd.DataFrame(index = blocks)
block_norm_cent_err = pd.DataFrame(index = blocks)
data_files_full = dict()
variability_sub_all_blocks = dict()
data_files = dict()
data_file_cent = dict()
upper_bound = dict()
lower_bound = dict()
error_dict_full = dict()
error_dict_block = dict()
block_er_mean = pd.DataFrame(index = blocks)
block_er_mean_norm = pd.DataFrame(index = blocks)
block_er_sem = pd.DataFrame(index = blocks)
block_er_sem_norm = pd.DataFrame(index = blocks)
anova_file_full = []
final_VAR_file = dict()
an_learn = []
an_learn_cond=[]
an_learn_sub = []
an_learn_nn = []
data_files_abs = dict()
learn_dict = dict()
learn_full = []
learn_df = pd.DataFrame()
data_files_absRT = dict()
data_file_absRT_sub  = dict()
total_correct_df = pd.DataFrame()
timeDuration_df = pd.DataFrame()

for file in filepaths:   
    files = os.listdir(filepath + file +'/')
    files_except_outliers = [i for i in files if i[:11] not in outliers]
    final = pd.DataFrame(index = blocks)
    final_cent = pd.DataFrame(index = blocks)
    final_er_norm = pd.DataFrame(index = blocks)
    final_er = pd.DataFrame(index = blocks)
    final_er_full = pd.DataFrame(index = ['all'])
    final_abs = pd.DataFrame(index = blocks)
    final_var = pd.DataFrame(index = blocks)
    total_correct = []
    timeDuration = []

    for indexx,i in enumerate(files_except_outliers):
        anova_file = pd.DataFrame()
        filename = i[:13]
        if filename[-1] == '1':
            skip_r = 2957
        else: skip_r = 2896

        df = pd.read_csv(filepath + file +'/' + i, skiprows=skip_r, delimiter = "\t")
        df1 = df[df["Code"].str.len()>5] #to seperate nan and isi from block

        df1[["Block", "Condition", "Target Button", "Trial Number"]] = df1["Code"].str.split(';',expand=True)
        df1.drop("Code", axis=1, inplace = True) 
        cols = ["Block", "Condition", "Target Button", "Trial Number"]

        for col in reversed(cols):
            temp = df1[col].str.strip()
            df1.drop(labels=[col], axis=1, inplace= True)
            df1.insert(1, col, temp)

        cols.remove("Condition")
        df1 = df1[df1['Block'] != 'PRACTICE BLOCK'] #removing practice blocks 
        for col in cols:
            df1[col] = df1[col].str.extract('(\d+)').astype(int) #extract numbers from string 
        df1["RT"] = df1["RT"]/10 
        ## nitshe paper, 2003, removing trials
        incorrect = df1[df1["Type"]=="incorrect"]
        hit = df1[df1["Type"]=="hit"]
        blockwise_before_drop = hit.groupby("Block").size().reset_index()
        blockwise_before_drop.drop('Block', axis=1, inplace = True)
        final_er['er_'+filename] = 1 - (blockwise_before_drop.to_numpy() / 120)
        final_er_full['er_'+filename] = 1-(len(hit) / len(df1))
        total_correct.append(len(hit))
        time = hit["Time"].tolist()
        timeDuration.append(time[-1]/60/10000)

        RT_mean = hit["RT"].mean()
        RT_std = hit["RT"].std()
        hit.drop(hit[(hit["RT"]> (RT_mean + 3*RT_std))].index, inplace=True)
        blockwise = hit.groupby("Block")["RT"].mean().reset_index()
        size = hit.groupby("Block").size().reset_index()
        blockwise['normal'] = blockwise['RT']/blockwise.iloc[0]['RT']
        final['norm_'+filename] = blockwise['normal'].to_numpy()

            ## PLOTTING INDIVIDUAL performance across the three conditions        
        # plt.figure(indexx)
        # plt.plot(blockwise["normal"],label = file)
        # plt.legend()
        # plt.ylabel('RT')
        # plt.title(filename)
        # plt.savefig("E:\\rajat\python_plots\ind_files\\ "+ filename + ".jpg")
        
        #implicit learning i.e., Block 6-Block5
        learn = blockwise.iloc[5]['normal'] - blockwise.iloc[4]['normal']  ## change for block
        learn_nn = blockwise.iloc[7]['RT'] - blockwise.iloc[2]['RT']
        norm_cent = []

        #analysing percent increse across blocks
        for index, row in blockwise.iterrows():
            #print(blockwise.iloc[index-1]['RT'])
            if index == 0:
                norm_cent.append(0)
            else:
                # norm_cent.append((float(row["RT"])-float(blockwise.iloc[index-1]['RT']))/float(row['RT']))
                norm_cent.append((float(row["RT"])-float(blockwise.iloc[index-1]['RT']))/abs(float(blockwise.iloc[index-1]['RT'])))

        blockwise["norm_cent"] = norm_cent
        blockwise['var'] =  statistics.variance(blockwise['RT'])
        blockwise['var_norm'] =  abs(blockwise['normal']-(blockwise['normal'].mean()))
        final_var[filename] =blockwise['var'].to_numpy()
        blockwise['ER'] = final_er.mean(axis = 1).to_numpy()
        blockwise['ER_norm'] = blockwise['ER']/blockwise.iloc[0]['ER']
        final_er_norm['er_norm_'+filename] = blockwise['ER_norm'].to_numpy()
        final_abs['rt_abs_'+filename] = blockwise['RT'].to_numpy()
        final_cent["RT_change"] = blockwise["norm_cent"].to_numpy()
        an_learn.append(learn)
        an_learn_nn.append(learn_nn)
        an_learn_cond.append(file)
        an_learn_sub.append(filename[-4:-2])        

        learn_df["subject"] =[filename[-4:-2]]
        learn_df['learn'] = learn
        learn_df["learn_nn"] = learn_nn
        learn_df["cond"] = file
        learn_full.append(learn_df)
        
        
        anova_file['subject'] = [filename[-4:-2]]*len(blocks)
        anova_file['rt_abs'] = blockwise['RT'].to_numpy()
        anova_file['norm_rt'] = blockwise['normal'].to_numpy()
        anova_file['er'] = blockwise['ER'].to_numpy()
        anova_file['var'] = blockwise['var'].to_numpy()
        anova_file['var_norm'] = blockwise['var_norm'].to_numpy()
        anova_file['block'] = blocks
        anova_file['condition'] = file
        anova_file['norm_cent'] = blockwise["norm_cent"]
        anova_file_full.append(anova_file)



        
    q1 = final.quantile(.25, axis = 1)
    q3  = final.quantile(.75, axis = 1)
    iqr = q3-q1
    upper_bound[file] = q3 + (1.5 * iqr)
    lower_bound[file] = q1 - (1.5 * iqr)
    data_files[file] = final.loc[['Block5','Block6','Block8']].T
    data_files_absRT[file] = final_abs.loc[['Block5','Block6','Block8']].T
    data_files_full[file] = final
    data_file_absRT_sub[file] = final_abs
    final_VAR_file[file] = final_var
    data_file_cent[file] = final_cent 
    block_mean[file] = final.mean(axis=1)
    block_norm_cent_mean[file] = final_cent.mean(axis = 1)
    block_er_mean_norm[file] = final_er_norm.mean(axis=1)
    block_rt_abs_mean[file] = final_abs.mean(axis=1)
    block_std[file] = final.std(axis=1)
    block_rt_abs_std[file] = final_abs.std(axis=1)
    block_err[file] = final.sem(axis=1) #standard error of mean
    block_norm_cent_err[file] =final_cent.sem(axis = 1)
    block_er_sem_norm[file] = final_er_norm.sem(axis=1)
    block_rt_abs_er[file] = final_abs.sem(axis=1)
    error_dict_block[file] = final_er
    block_er_mean[file] = final_er.mean(axis = 1)
    block_er_sem[file] = final_er.sem(axis = 1)
    error_dict_full[file] = final_er_full
    data_files_abs[file] = final_abs
    total_correct_df[file] = total_correct
    timeDuration_df[file] = timeDuration

Generating csvs for ANOVA

In [ ]:
an_file = pd.concat(anova_file_full)
learn_file = pd.concat(learn_full)
anova_learn = pd.DataFrame({'Subject': an_learn_sub, 'learning':an_learn, 'learning_nn': an_learn_nn, 'condition': an_learn_cond})
# an_file.to_csv(r"E:/rajat/laptop_files/dissertation/SRTT_CSV/an_file_less_1mA.csv") 

In [ ]:
learn_cond = anova_learn.groupby('condition')['learning'].apply(list).to_dict()
learn_cond_nn = anova_learn.groupby('condition')['learning_nn'].apply(list).to_dict()
er_ = an_file.groupby('block')['er'].apply(list).to_dict()
# learn_cond.set_index("condition")
anova_learn.set_index("Subject",drop = "index", inplace = True)
anova_learn.sort_index(inplace = True)
# anova_learn.to_csv(r"E:/rajat/laptop_files/dissertation/SRTT_CSV/anova_learn.csv")

Total correct trials for each tDCS condition 

In [ ]:
for i, row in total_correct_df.T.iterrows():
    print(i + " mean:" + str(row.mean()))
    print(i + " stdDev:" + str(row.std()))
    print(i + " min :" + str(min(row)))
    print(i + " max :" + str(max(row)))

Total stimulation time for each tDCS condition

In [ ]:
for i, row in timeDuration_df.T.iterrows():
    print(i + ": " + str(row.mean()))
    print(i + ": " + str(row.std()))

Violin Plots
RT distribution: Fixed vs Individualised dose 

In [ ]:
aa = data_files_full["Fixed dose"]
bb = data_files_full["Individualised dose"]
sub_list_ind = []
for i, row in bb.T.iterrows():
    sub_list_ind.append(i)

sub_list_fix = []
for i, row in aa.T.iterrows():
    sub_list_fix.append(i)


In [ ]:
data_lf = pd.melt(aa, value_vars=sub_list_fix, var_name = "sub", value_name="RT")
data_lf["Blocks"] = ["Block1","Block2", "Block3", "Block4", "Block5", "Block6", "Block7", "Block8"]*21
data_lf["cond"] = ["Fixed"]*len(data_lf)  

In [ ]:
data_lf2 = pd.melt(bb, value_vars=sub_list_ind, var_name = "sub", value_name="RT")
data_lf2["Blocks"] = ["Block1","Block2", "Block3", "Block4", "Block5", "Block6", "Block7", "Block8"]*21
data_lf2["cond"] = ["Individualised"]*len(data_lf)

In [ ]:
df_plot = pd.concat([data_lf, data_lf2])

In [ ]:
show_legend = [True,False,False,False, False, False,False, False]

fig = go.Figure()
for i in range(0,len(pd.unique(df_plot['Blocks']))):

    fig.add_trace(go.Violin(x= df_plot['Blocks'][(df_plot['cond'] == 'Fixed') &
                                    (df_plot['Blocks'] == pd.unique(df_plot['Blocks'])[i])],
                        y=df_plot['RT'][(df_plot['cond'] == 'Fixed')&
                                            (df_plot['Blocks'] == pd.unique(df_plot['Blocks'])[i])],
                                            
                                            legendgroup='Fixed', scalegroup='Fixed', name='Fixed',
                        side='negative',
                        pointpos=-1.07, # where to position points
                        line_color='lightseagreen',
                        showlegend=show_legend[i]
                        ))
            

    fig.add_trace(go.Violin(x=df_plot['Blocks'][(df_plot['cond'] == 'Individualised') &
                                    (df_plot['Blocks'] == pd.unique(df_plot['Blocks'])[i])],
                        y=df_plot['RT'][(df_plot['cond'] == 'Individualised')&
                                            (df_plot['Blocks'] == pd.unique(df_plot['Blocks'])[i])],
                        legendgroup='Individualised', scalegroup='Individualised', name='Individualised',
                        side='positive',
                        pointpos=1.1, # where to position points
                        line_color='mediumpurple',
                        showlegend=show_legend[i]                        
                        ))
                        


# update characteristics shared by all traces
fig.update_traces(meanline_visible=True,
                  points='all', # show all points
                  jitter=0.05,  # add some jitter on points for better visibility
                  scalemode='count') #scale violin plot area with total count
fig.update_layout(
    title_text="Behavioural Performance",
    # xaxis_title = "_",\
    yaxis_title = 'Normalised RT',
    violingap=0.3, violingroupgap=0, violinmode='overlay',
    autosize = False,
    width = 1300, 
    height = 650,
    font_size = 20 )
fig.write_image("/home/friction_15/lab/dissertation/reviewFigures/bimodal.jpg", scale = 10)
fig.show()

Analysing the baseline performance

In [ ]:
b1_sham_abs_gtr_1mA =data_files_abs['Sham'].iloc[0]  #sham cond baseline >1mA

In [ ]:
b1_sham_abs_less_1mA = data_files_abs['Sham'].iloc[0] #sham condi baseline <1

In [ ]:
test, p = shapiro(b1_sham_abs_gtr_1mA) #testing for normality
print(test, p)

In [ ]:
print(stats.ttest_ind(b1_sham_abs_gtr_1mA, b1_sham_abs_less_1mA, equal_var= False)) #t-test between the >1mA and <1mA 

In [ ]:
# fig = plt.figure(figsize=(15,10))
fig,ax = plt.subplots(figsize = ((15,10)))
data_to_plot = [b1_sham_abs_gtr_1mA, b1_sham_abs_less_1mA]
plt.boxplot(data_to_plot, patch_artist=True)
labels = [item.get_text() for item in ax.get_xticklabels()]
labels[1] = '<1mA'
labels[0] = '>1mA'
ax.set_xticklabels(labels)
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)
ax.spines.left.set_visible(False)
ax.spines.bottom.set_visible(False)
plt.yticks(size = 18)
plt.xticks(size = 18)
plt.grid(True, axis = "y", color='w', linestyle='-', linewidth=2)
plt.gca().patch.set_facecolor('.91')

plt.ylim(250,650)
plt.rc('xtick', labelsize=23) 
plt.rc('ytick', labelsize=23) 
plt.rc('axes', titlesize=30)   
plt.rc('axes', labelsize=22)
plt.xlabel('Stimulation dose (mA)', fontsize = 26)
plt.ylabel('Reaction time (ms)', fontsize = 26)
plt.title('BASELINE PERFORMANCE - Sham Block1', fontsize = 30)
plt.show()
# fig.savefig(r"/home/friction_15/lab/dissertation/reviewFigures/baselinePerformance.jpg", format="jpg", dpi = 500)

subjectwise - block rxn time - learning curve

In [ ]:
## Absolute Reaction time
ax = block_rt_abs_mean.plot(yerr = block_rt_abs_er, ylim=[300, 500],figsize = (19,11), rot = 25, capsize = 5, linewidth = 2.3, 
    marker = 'o', markersize = 12, markerfacecolor = 'white' )

ax.legend(loc = "upper right", borderpad=1.5, labelspacing=1.5, handlelength=5, prop = {"size":12 })
# ax.set_frame_on(False)
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)
ax.spines.left.set_visible(False)
ax.spines.bottom.set_visible(False)
plt.yticks(size = 18)
plt.xticks(size = 18)
plt.grid(True, axis = "y", color='w', linestyle='-', linewidth=2)
plt.gca().patch.set_facecolor('.91')
plt.ylabel("RT(ms)", fontname = "sans", size = 26)
plt.xlabel("Blocks", fontname = "sans", size = 26)
# plt.title("Stimulation intensity >1mA", fontname= "serif", size = 30)
# plt.savefig(r"/home/friction_15/lab/dissertation/reviewFigures/srtt/Abs.jpg", format="jpg", dpi = 500)
# plt.savefig(r"/home/friction_15/lab/dissertation/reviewFigures/srttgtr/gtrAbsRT.jpg", format="jpg", dpi = 500)
# plt.savefig(r'E:\new_figures\rt_block_21.jpeg')

In [ ]:
## Normalised RT

ax = block_mean.plot(yerr = block_err, ylim=[0.8, 1.10],figsize = (19,11), rot = 25, capsize = 5, linewidth = 2.3, 
    marker = 'o', markersize = 12, markerfacecolor = 'white' ) #for poster linewidth = 4 otherwise 2.3; paper: figsize = (19,11)
ax.legend(loc = "upper right", borderpad=1.5, labelspacing=1.5, handlelength=5, prop = {"size":15})
ax.spines.right.set_visible(True)
ax.spines.top.set_visible(True)
ax.spines.left.set_visible(True)
ax.spines.bottom.set_visible(True)
plt.yticks(size = 20)
plt.xticks(size = 20)
plt.grid(True, axis = "y", color='grey', linestyle='-', linewidth=0.8)
plt.gca().patch.set_facecolor('1') #check from previous cell for paper
plt.ylabel("Normalised RT", fontname = "sans",size = 30)
plt.xlabel("Blocks", fontname = "sans", size = 30)
# plt.savefig(r"/home/friction_15/lab/dissertation/reviewFigures/srtt/normPoster.jpg", format="jpg", dpi = 500)

# plt.title("Stimulation intensity <1mA", fontname = "serif",size = 30)
# plt.savefig(r"/home/friction_15/lab/dissertation/reviewFigures/srttLess/lessnormRT.jpg", format="jpg", dpi = 500)

Z-score

In [ ]:
# inter subject z- score
for key in data_files_full:
    mean_block_1 = data_files_full[key].loc["Block2"].mean()
    z_score = (data_files_full[key].T - data_files_full[key].T.mean()) / data_files_full[key].T.std()
    std_blockwise = data_files_full[key].std(axis = 1)
    fig = px.strip(z_score, hover_name = z_score.index.str[-4:-2])
    fig.show()

Normaility test

In [ ]:
#shapiro wilk test
for blocks in data_files_full:
    normality_full, p_val = shapiro(data_files_full[blocks])
    print(blocks, "test_stat- ", normality_full, "p_value- ", p_val )

In [ ]:
## Blockwise Distribution across tDCS conditions
fig1, axis1 = plt.subplots(2,4,figsize=(15, 15))
fig2, axis2 = plt.subplots(2,4,figsize=(15, 15))
fig3, axis3 = plt.subplots(2,4,figsize=(15, 15))
axis = [axis1,axis2,axis3]
for ind, blocks in enumerate(data_files_abs):
    k = 0
    i = 0
    j = 0
    for index, row in data_files_full[blocks].iterrows():
        normality_rt, p = shapiro(row)
        print(blocks,index, ': ', 'test_stat - ', normality_rt, 'p-value - ',p)
        row.plot.hist(ax = axis[ind][i,j], title = index)
        k += 1
        j = k%4
        if k>3:
            i = 1